# Main notebook - experimenting with guesses

This does semantic search based solely on repository name and user input.

Compare to
[`findrepo`](https://github.com/EliahKagan/newrepo-findrepo#using-findrepo).

In [1]:
import logging
from types import SimpleNamespace

import numpy as np
from tabulate import tabulate

import fr2ex

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
names = fr2ex.remote.fetch_repo_names()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [4]:
len(names)

678

In [5]:
moderation = fr2ex.moderation.get_moderation(names)

INFO:root:Querying OpenAI moderation endpoint.


In [6]:
len(moderation)

678

In [7]:
any(result['flagged'] for result in moderation)

False

In [8]:
fr2ex.tokens.report_cost(names)

It looks like the rate is $0.0004 per 1000 tokens. If so, the cost to
process 1920 tokens is about $0.0007680 (that is, 0.0768000 cents).


In [9]:
embeddings = fr2ex.embedding.embed_many(names)

INFO:root:Querying OpenAI embeddings endpoint.


In [10]:
def guess(name: str, count: int = 5) -> None:
    """Show top guesses for similarity of name to already embedded names."""
    scores = embeddings @ fr2ex.embedding.embed(name)
    ordering = sorted(zip(scores, names), reverse=True)
    table = [(name, score) for score, name in ordering]
    return tabulate(table[:count], tablefmt='html', floatfmt='.6f')

In [11]:
guess('algorithm')

Calculator,0.868882
Geometry,0.865520
algorithms-suggestions,0.860273
Alias,0.851841
crystal-algo,0.849729


In [12]:
guess('sorting')

SortingRanges,0.932602
Sorts,0.909067
sortkey,0.859544
PartialSort,0.855028
tsort,0.840354


In [13]:
guess('algorithm-visualization')

algorithms-suggestions,0.889052
algorithms-python,0.859356
crystal-algo,0.856027
Graph-gist,0.842657
codegraph,0.833195


In [14]:
guess('maven')

try-maven,0.915987
mavener,0.880060
retry-maven,0.868202
pimpl,0.836270
try-gradle,0.833675


In [15]:
guess('find-repo')

newrepo-findrepo,0.916260
findrepo2-experiment,0.867309
backup-repos,0.865014
find-articles,0.857093
TestRepo,0.842355


In [16]:
guess('graph theory', count=10)

codegraph,0.881156
Graph,0.879349
Graph-gist,0.864826
Geometry,0.839664
wpf-graph,0.829071
gh-profile,0.814756
c-sketches,0.813895
trig,0.809151
Draw,0.808799
crystal-sketches,0.804894


In [17]:
guess('shortest paths', count=10)

Dijkstra,0.891414
Paths,0.838885
TreeTraversal,0.830637
Kruskal,0.816358
algorithms-suggestions,0.816185
Traverse,0.813017
Permutations,0.806546
treejs,0.801362
DFS,0.800804
kt-short,0.800315


In [18]:
guess('shortest-paths', count=10)

Dijkstra,0.892566
Paths,0.836328
TreeTraversal,0.836267
Traverse,0.819881
algorithms-suggestions,0.819237
Kruskal,0.814068
type-arrows,0.813011
kt-short,0.811268
sequence-dag,0.809361
dot-and-cross,0.806405


In [19]:
guess('graph traversal', count=10)

TreeTraversal,0.897430
Traverse,0.869958
Graph,0.868275
codegraph,0.856913
Graph-gist,0.851086
TraverseTree,0.850041
TreeTraversalAnimations,0.844913
Dijkstra,0.825562
DFS,0.823535
sequence-dag,0.819539


In [20]:
guess('graph-traversal', count=10)

TreeTraversal,0.904181
Traverse,0.879860
Graph,0.878266
Graph-gist,0.875246
codegraph,0.872211
TraverseTree,0.865436
TreeTraversalAnimations,0.851144
Dijkstra,0.842906
wpf-graph,0.840953
sequence-dag,0.837051


In [21]:
guess('graphics', count=10)

Graph,0.889289
printing,0.882984
Geometry,0.881471
Draw,0.875394
fps,0.859528
works,0.848890
coordinates,0.845145
Components,0.842697
interop,0.841286
Benchmark,0.840062


In [22]:
guess('deluge')

Flood,0.864917
foobar,0.825785
lambda-demo,0.819114
Dynamo,0.816751
yule,0.813776


In [23]:
guess('source control')

git,0.822978
Linkage,0.807195
gitscripts,0.806823
capture.net,0.803136
test-site,0.801780


In [24]:
guess('source-control')

git,0.831379
git-notes,0.830440
dot-imports,0.815820
gitscripts,0.812936
editor-configs,0.812632
